In [1]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model

In [7]:
with open("/content/sample_data/IMDB Dataset.csv", "r", encoding="utf-8", errors="ignore") as f:
    lines = f.readlines()

with open("cleaned_file.csv", "w", encoding="utf-8") as f:
    for line in lines:
        if line.count('"') % 2 != 0:  # Odd number of quotes
            line = line.replace('"', '')  # Remove quotes (or fix logic)
        f.write(line)

df = pd.read_csv("cleaned_file.csv")


In [8]:
print(df)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
def load_dataset():
    df = pd.read_csv('/content/sample_data/IMDB Dataset.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # Get English stop words
    english_stops = set(stopwords.words('english'))

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case

    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Reviews
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object 

Sentiment
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


/tmp/ipython-input-441948422.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_data = y_data.replace('negative', 0)


In [13]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)


Train Set
48578    [sat, watch, movie, thought, might, slightly, ...
33572    [many, us, find, art, agreeable, masterpiece, ...
43532    [hlots, outstanding, series, nypd, blue, never...
2102     [i, mixed, feelings, les, valseuses, written, ...
46584    [season, season, players, characters, show, ap...
                               ...                        
14703    [all, dogs, go, heaven, plays, canine, crimina...
45461    [a, klutzy, young, man, returns, west, schoole...
3800     [the, rap, book, movie, based, one, difficult,...
22914    [i, saw, star, movie, the, daily, show, though...
29546    [this, another, case, hollywood, arrogance, pr...
Name: review, Length: 40000, dtype: object 

43947    [my, watch, came, little, late, glad, watched,...
10308    [i, seen, movie, years, last, time, really, dr...
10147    [come, fantastic, drama, on, screen, are, joki...
28752    [any, time, movie, myopic, desire, present, pa...
41986    [this, one, best, non, english, series, i, see...
 

In [14]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

In [15]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)


Encoded X Train
 [[ 1694    34     3 ...     0     0     0]
 [   37    86    79 ... 28169  2721  3798]
 [39931  1304   112 ...     0     0     0]
 ...
 [    2  4450   176 ...     0     0     0]
 [    1   118   230 ...     0     0     0]
 [    8    73   324 ...     0     0     0]] 

Encoded X Test
 [[  215    34   291 ...     0     0     0]
 [    1    38     3 ...     0     0     0]
 [  119   715   349 ...     0     0     0]
 ...
 [  475   688     4 ... 17426    37   546]
 [   16     3     5 ...     0     0     0]
 [    7  1512     9 ...   177  2473  1791]] 

Maximum review length:  130


In [16]:
# ARCHITECTURE
EMBED_DIM = 32
LSTM_OUT = 64

model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(LSTM(LSTM_OUT))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model.summary())

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [17]:

checkpoint = ModelCheckpoint(
    'models/LSTM.h5',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)


In [18]:
model.fit(x_train, y_train, batch_size = 128, epochs = 5, callbacks=[checkpoint])

Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step - accuracy: 0.5185 - loss: 0.6907
Epoch 1: accuracy improved from -inf to 0.52403, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 67s 203ms/step - accuracy: 0.5186 - loss: 0.6906
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step - accuracy: 0.5855 - loss: 0.6669
Epoch 2: accuracy improved from 0.52403 to 0.60815, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 63s 201ms/step - accuracy: 0.5856 - loss: 0.6668
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step - accuracy: 0.6940 - loss: 0.5922
Epoch 3: accuracy improved from 0.60815 to 0.70807, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 82s 200ms/step - accuracy: 0.6941 - loss: 0.5922
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step - accuracy: 0.7198 - loss: 0.5748
Epoch 4: accuracy did not improve from 0.70807
313/313 ━━━━━━━━━━━━━━━━━━━━ 62s 199ms/step - accuracy: 0.7197 - loss: 0.5749
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step - accuracy: 0.7143 - loss: 0.5454
Epoch 5: accuracy improved from 0.70807 to 0.71573, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 82s 197ms/step - accuracy: 0.7143 - loss: 0.5454


In [20]:
y_pred_prob = model.predict(x_test, batch_size = 128)
y_pred = (y_pred_prob > 0.5).astype(int)

true = 0
for i, y in enumerate(y_test):
    if y == y_pred[i]:
        true += 1

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred) - true))
print('Accuracy: {}'.format(true/len(y_pred)*100))

79/79 ━━━━━━━━━━━━━━━━━━━━ 7s 84ms/step
Correct Prediction: 8028
Wrong Prediction: 1972
Accuracy: 80.28


In [21]:
loaded_model = load_model('models/LSTM.h5')

In [22]:
review = str(input('Movie Review: '))

Movie Review: Nothing was typical about this. Everything was beautifully done in this movie, the story, the flow, the scenario, everything. I highly recommend it for mystery lovers, for anyone who wants to watch a good movie!


In [25]:
# Pre-process input
import re
from nltk.corpus import stopwords
english_stops = set(stopwords.words('english'))

regex = re.compile(r'[^a-zA-Z\s]')
review = regex.sub('', review)
print('Cleaned: ', review)

words = review.split(' ')
filtered = [w for w in words if w not in english_stops]
filtered = ' '.join(filtered)
filtered = [filtered.lower()]

print('Filtered: ', filtered)

Cleaned:  Nothing was typical about this Everything was beautifully done in this movie the story the flow the scenario everything I highly recommend it for mystery lovers for anyone who wants to watch a good movie
Filtered:  ['nothing typical everything beautifully done movie story flow scenario everything i highly recommend mystery lovers anyone wants watch good movie']


In [26]:
tokenize_words = token.texts_to_sequences(filtered)
tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
print(tokenize_words)

[[  76  695  174 1238  126    3   14 2719 2486  174    1  446  281  656
  1670  157  393   34    9    3    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]]


In [27]:
result = loaded_model.predict(tokenize_words)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step
[[0.8174888]]


In [28]:
if result >= 0.7:
    print('positive')
else:
    print('negative')

positive
